In [20]:
!pip install pyspark
!pip install -q findspark
import findspark
findspark.init()

In [21]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("SkewinDF").getOrCreate()
spark

Loading Data Skew

In [22]:
import numpy as np
import pandas as pd
import random

key_1=[101] * 1
key_2=[201] * 8
key_3=[301] * 3
key_4=[401] * 2
print(key_1)

OrderID = key_1 + key_2 + key_3 + key_4

random.shuffle(OrderID)

Qty_1 = list(np.random.randint(low=1,high=100,size=len(key_1)))
Qty_2 = list(np.random.randint(low=1,high=200,size=len(key_2)))
Qty_3 = list(np.random.randint(low=1,high=1000,size=len(key_3)))
Qty_4 = list(np.random.randint(low=1,high=50,size=len(key_4)))
Qty = Qty_1 + Qty_2 + Qty_3 + Qty_4

Sales_1 = list(np.random.randint(low=10, high=100,size=len(key_1)))
Sales_2 = list(np.random.randint(low=50, high=3400,size=len(key_2)))
Sales_3 = list(np.random.randint(low=12, high=2000,size=len(key_3)))
Sales_4 = list(np.random.randint(low=40,high=1000,size=len(key_4)))
Sales = Sales_1 + Sales_2 + Sales_3 + Sales_4

Discount = list(np.random.randint(low=0,high=2,size=len(OrderID)))
data1=list(zip(OrderID,Qty,Sales,Discount))

#Create the Pandas DF
data_skew=pd.DataFrame(data1,columns=['OrderID','Qty','Sales','Discount'])

data2=[[1,101,'pencil',4.99],
       [2,101,'book',9.5],
       [3,101,'scissors',14],
       [4,301,'glue',7],
       [5,201,'marker',8.49],
       [6,301,'label',2],
       [7,201,'calculator',3.99],
       [8,501,'eraser',1.55],
       ]
data_small = pd.DataFrame(data2,columns=["ProductID","OrderID","Product","Price"])
data_small

[101]


,ProductID,OrderID,Product,Price
0,1,101,pencil,4.99
1,2,101,book,9.50
2,3,101,scissors,14.00
3,4,301,glue,7.00
4,5,201,marker,8.49
5,6,301,label,2.00
6,7,201,calculator,3.99
7,8,501,eraser,1.55


In [23]:
#Create DF From Pandas
spark.conf.set("spark.sql.execution.arrow.enabled","true")
df_skew = spark.createDataFrame(data_skew)
df_skew.printSchema()
df_skew.show()
df_skew.rdd.getNumPartitions()

root
 |-- OrderID: long (nullable = true)
 |-- Qty: long (nullable = true)
 |-- Sales: long (nullable = true)
 |-- Discount: long (nullable = true)

+-------+---+-----+--------+
|OrderID|Qty|Sales|Discount|
+-------+---+-----+--------+
|    201| 66|   70|       1|
|    301|137| 2025|       1|
|    201|147| 3056|       0|
|    101|187| 1691|       0|
|    201|198|  548|       1|
|    201| 81| 3309|       0|
|    201|193|   56|       0|
|    201| 91| 3350|       0|
|    301|128| 2929|       0|
|    201|930| 1408|       1|
|    401| 25|  818|       1|
|    201|121| 1164|       1|
|    301| 42|  505|       1|
|    401| 37|  755|       1|
+-------+---+-----+--------+



2

In [24]:
df_small=spark.createDataFrame(data_small)
df_small.printSchema()
df_small.show()
df_small.rdd.getNumPartitions()

root
 |-- ProductID: long (nullable = true)
 |-- OrderID: long (nullable = true)
 |-- Product: string (nullable = true)
 |-- Price: double (nullable = true)

+---------+-------+----------+-----+
|ProductID|OrderID|   Product|Price|
+---------+-------+----------+-----+
|        1|    101|    pencil| 4.99|
|        2|    101|      book|  9.5|
|        3|    101|  scissors| 14.0|
|        4|    301|      glue|  7.0|
|        5|    201|    marker| 8.49|
|        6|    301|     label|  2.0|
|        7|    201|calculator| 3.99|
|        8|    501|    eraser| 1.55|
+---------+-------+----------+-----+



2

# Run a shuffle join with small sized data

In [25]:
joined_df = df_skew.join(df_small,df_skew.OrderID==df_small.OrderID,how="inner")
joined_df.show()

+-------+---+-----+--------+---------+-------+----------+-----+
|OrderID|Qty|Sales|Discount|ProductID|OrderID|   Product|Price|
+-------+---+-----+--------+---------+-------+----------+-----+
|    201| 66|   70|       1|        7|    201|calculator| 3.99|
|    201| 66|   70|       1|        5|    201|    marker| 8.49|
|    301|137| 2025|       1|        6|    301|     label|  2.0|
|    301|137| 2025|       1|        4|    301|      glue|  7.0|
|    201|147| 3056|       0|        7|    201|calculator| 3.99|
|    201|147| 3056|       0|        5|    201|    marker| 8.49|
|    101|187| 1691|       0|        3|    101|  scissors| 14.0|
|    101|187| 1691|       0|        2|    101|      book|  9.5|
|    101|187| 1691|       0|        1|    101|    pencil| 4.99|
|    201|198|  548|       1|        7|    201|calculator| 3.99|
|    201|198|  548|       1|        5|    201|    marker| 8.49|
|    201| 81| 3309|       0|        7|    201|calculator| 3.99|
|    201| 81| 3309|       0|        5|  

In [26]:
# df increases the partition number to 200 automatically when spark performs data shuffling(join,aggregation)
print(joined_df.rdd.getNumPartitions())
joined_df.rdd.glom().collect()

2


[[Row(OrderID=201, Qty=66, Sales=70, Discount=1, ProductID=7, OrderID=201, Product='calculator', Price=3.99),
  Row(OrderID=201, Qty=66, Sales=70, Discount=1, ProductID=5, OrderID=201, Product='marker', Price=8.49),
  Row(OrderID=301, Qty=137, Sales=2025, Discount=1, ProductID=6, OrderID=301, Product='label', Price=2.0),
  Row(OrderID=301, Qty=137, Sales=2025, Discount=1, ProductID=4, OrderID=301, Product='glue', Price=7.0),
  Row(OrderID=201, Qty=147, Sales=3056, Discount=0, ProductID=7, OrderID=201, Product='calculator', Price=3.99),
  Row(OrderID=201, Qty=147, Sales=3056, Discount=0, ProductID=5, OrderID=201, Product='marker', Price=8.49),
  Row(OrderID=101, Qty=187, Sales=1691, Discount=0, ProductID=3, OrderID=101, Product='scissors', Price=14.0),
  Row(OrderID=101, Qty=187, Sales=1691, Discount=0, ProductID=2, OrderID=101, Product='book', Price=9.5),
  Row(OrderID=101, Qty=187, Sales=1691, Discount=0, ProductID=1, OrderID=101, Product='pencil', Price=4.99),
  Row(OrderID=201, Qty=

In [27]:
spark.conf.set("spark.sql.shuffle.partitions",8)

# run_join()

joined_df = df_skew.join(df_small,df_skew.OrderID==df_small.OrderID,how="inner")
joined_df.show

print(joined_df.rdd.getNumPartitions())
joined_df.rdd.glom().collect()

2


[[Row(OrderID=201, Qty=66, Sales=70, Discount=1, ProductID=7, OrderID=201, Product='calculator', Price=3.99),
  Row(OrderID=201, Qty=66, Sales=70, Discount=1, ProductID=5, OrderID=201, Product='marker', Price=8.49),
  Row(OrderID=301, Qty=137, Sales=2025, Discount=1, ProductID=6, OrderID=301, Product='label', Price=2.0),
  Row(OrderID=301, Qty=137, Sales=2025, Discount=1, ProductID=4, OrderID=301, Product='glue', Price=7.0),
  Row(OrderID=201, Qty=147, Sales=3056, Discount=0, ProductID=7, OrderID=201, Product='calculator', Price=3.99),
  Row(OrderID=201, Qty=147, Sales=3056, Discount=0, ProductID=5, OrderID=201, Product='marker', Price=8.49),
  Row(OrderID=101, Qty=187, Sales=1691, Discount=0, ProductID=3, OrderID=101, Product='scissors', Price=14.0),
  Row(OrderID=101, Qty=187, Sales=1691, Discount=0, ProductID=2, OrderID=101, Product='book', Price=9.5),
  Row(OrderID=101, Qty=187, Sales=1691, Discount=0, ProductID=1, OrderID=101, Product='pencil', Price=4.99),
  Row(OrderID=201, Qty=

In [28]:
#Perform a join() and descriptive statistics on a skewd data
from pyspark.sql.functions import *

groups=df_skew.join(df_small,df_skew.OrderID==df_small.OrderID,how="inner")

summary=groups.select(mean(groups.Sales).alias('Avg_sales'),
                      stddev(groups.Sales).alias('STD_sales'),
                      min(groups.Sales).alias('Min_sales'),
                      max(groups.Sales).alias('max_sales'))
summary.show()

+---------+------------------+---------+---------+
|Avg_sales|         STD_sales|Min_sales|max_sales|
+---------+------------------+---------+---------+
|  1676.52|1198.8445173026678|       56|     3350|
+---------+------------------+---------+---------+



#Mitigate a skewness : SALTING

In [29]:
from pyspark.sql.functions import *

#Adding random values [0 2]
df_skew_salting = df_skew.withColumn("_salt_",round(rand()*2))
df_small_salting = df_small.withColumn("_salt_",round(rand()*2))

df_skew_salting.show()
df_small_salting.show()

+-------+---+-----+--------+------+
|OrderID|Qty|Sales|Discount|_salt_|
+-------+---+-----+--------+------+
|    201| 66|   70|       1|   2.0|
|    301|137| 2025|       1|   2.0|
|    201|147| 3056|       0|   1.0|
|    101|187| 1691|       0|   2.0|
|    201|198|  548|       1|   2.0|
|    201| 81| 3309|       0|   1.0|
|    201|193|   56|       0|   0.0|
|    201| 91| 3350|       0|   0.0|
|    301|128| 2929|       0|   0.0|
|    201|930| 1408|       1|   1.0|
|    401| 25|  818|       1|   1.0|
|    201|121| 1164|       1|   0.0|
|    301| 42|  505|       1|   1.0|
|    401| 37|  755|       1|   0.0|
+-------+---+-----+--------+------+

+---------+-------+----------+-----+------+
|ProductID|OrderID|   Product|Price|_salt_|
+---------+-------+----------+-----+------+
|        1|    101|    pencil| 4.99|   2.0|
|        2|    101|      book|  9.5|   1.0|
|        3|    101|  scissors| 14.0|   2.0|
|        4|    301|      glue|  7.0|   1.0|
|        5|    201|    marker| 8.49|   1.0|

In [30]:
#Rpartition using __salt__:

df_skew_salting=df_skew_salting.repartition(3,"_salt_")
df_small_salting=df_small_salting.repartition(3,"_salt_")

print(df_skew_salting.rdd.glom().collect())

[[Row(OrderID=201, Qty=66, Sales=70, Discount=1, _salt_=2.0), Row(OrderID=301, Qty=137, Sales=2025, Discount=1, _salt_=2.0), Row(OrderID=101, Qty=187, Sales=1691, Discount=0, _salt_=2.0), Row(OrderID=201, Qty=198, Sales=548, Discount=1, _salt_=2.0)], [Row(OrderID=201, Qty=193, Sales=56, Discount=0, _salt_=0.0), Row(OrderID=201, Qty=91, Sales=3350, Discount=0, _salt_=0.0), Row(OrderID=301, Qty=128, Sales=2929, Discount=0, _salt_=0.0), Row(OrderID=201, Qty=121, Sales=1164, Discount=1, _salt_=0.0), Row(OrderID=401, Qty=37, Sales=755, Discount=1, _salt_=0.0)], [Row(OrderID=201, Qty=147, Sales=3056, Discount=0, _salt_=1.0), Row(OrderID=201, Qty=81, Sales=3309, Discount=0, _salt_=1.0), Row(OrderID=201, Qty=930, Sales=1408, Discount=1, _salt_=1.0), Row(OrderID=401, Qty=25, Sales=818, Discount=1, _salt_=1.0), Row(OrderID=301, Qty=42, Sales=505, Discount=1, _salt_=1.0)]]


In [31]:
print(df_small_salting.rdd.glom().collect())

[[Row(ProductID=1, OrderID=101, Product='pencil', Price=4.99, _salt_=2.0), Row(ProductID=3, OrderID=101, Product='scissors', Price=14.0, _salt_=2.0)], [Row(ProductID=7, OrderID=201, Product='calculator', Price=3.99, _salt_=0.0)], [Row(ProductID=2, OrderID=101, Product='book', Price=9.5, _salt_=1.0), Row(ProductID=4, OrderID=301, Product='glue', Price=7.0, _salt_=1.0), Row(ProductID=5, OrderID=201, Product='marker', Price=8.49, _salt_=1.0), Row(ProductID=6, OrderID=301, Product='label', Price=2.0, _salt_=1.0), Row(ProductID=8, OrderID=501, Product='eraser', Price=1.55, _salt_=1.0)]]


In [32]:
# apply join ()

df_skew_salting.drop("_salt_")
df_small_salting.drop("_salt_")

groups=df_skew.join(df_small_salting,df_skew_salting.OrderID==df_small_salting.OrderID,how="inner")

summary=groups.select(mean(groups.Sales).alias('Avg_sales'),
                      stddev(groups.Sales).alias('STD_sales'),
                      min(groups.Sales).alias('Min_sales'),
                      max(groups.Sales).alias('max_sales'))
summary.show()

+---------+------------------+---------+---------+
|Avg_sales|         STD_sales|Min_sales|max_sales|
+---------+------------------+---------+---------+
|  1676.52|1198.8445173026678|       56|     3350|
+---------+------------------+---------+---------+



# Run a shuffle join() to see how the skew effects computation resources



In [33]:
#Perform a join() and descriptive statistics on a skewd data
from pyspark.sql.functions import *

spark.conf.set("spark.sql.shuffle.partitions",8)

groups=df_skew.join(df_small,df_skew.OrderID==df_small.OrderID,how="inner")

summary=groups.select(mean(groups.Sales).alias('Avg_sales'),
                      stddev(groups.Sales).alias('STD_sales'),
                      min(groups.Sales).alias('Min_sales'),
                      max(groups.Sales).alias('max_sales'))
summary.show()

+---------+------------------+---------+---------+
|Avg_sales|         STD_sales|Min_sales|max_sales|
+---------+------------------+---------+---------+
|  1676.52|1198.8445173026678|       56|     3350|
+---------+------------------+---------+---------+



#Mitigating data skewness : salting



In [34]:
from pyspark.sql.functions import *

spark.conf.set("spark.sql.shuffle.partitions",8)

from pyspark.sql.functions import *

#Adding random values [0 7]
df_skew_salting = df_skew.withColumn("_salt_",round(rand()*7))
df_small_salting = df_small.withColumn("_salt_",round(rand()*7))

#Repartition using _salt__
df_skew_salting = df_skew_salting.repartition(8,"_salt_")
df_small_salting = df_small_salting.repartition(8,"_salt_")

# drop salting
df_skew_salting.drop("_salt_")
df_small_salting.drop("_salt_")

#apply Join
groups = df_skew_salting.join(df_small_salting,df_skew_salting.OrderID==df_small_salting.OrderID,how="inner")

summary=groups.select(mean(groups.Sales).alias("Avg(sales)"),
                      stddev(groups.Sales).alias("STD(sales)"),
                      min(groups.Sales).alias("MIN(sales)"),
                      max(groups.Sales).alias("MAX(sales)"),
)
summary.show()

+----------+------------------+----------+----------+
|Avg(sales)|        STD(sales)|MIN(sales)|MAX(sales)|
+----------+------------------+----------+----------+
|   1676.52|1198.8445173026678|        56|      3350|
+----------+------------------+----------+----------+



# Mitigate data skewness : Broadcast hash join

In [35]:
from pyspark.sql.functions import *

groups_brd = df_skew.join(broadcast(df_small), df_skew.OrderID == df_small.OrderID,how='inner')

summary=groups_brd.select(mean(groups.Sales).alias("Avg(sales)"),
                      stddev(groups.Sales).alias("STD(sales)"),
                      min(groups.Sales).alias("MIN(sales)"),
                      max(groups.Sales).alias("MAX(sales)"),
)
summary.show()


+----------+------------------+----------+----------+
|Avg(sales)|        STD(sales)|MIN(sales)|MAX(sales)|
+----------+------------------+----------+----------+
|   1676.52|1198.8445173026678|        56|      3350|
+----------+------------------+----------+----------+



# Mitigate data skewness : AQE (spark V3.x)

In [36]:
spark.conf.set("spark.sql.adaptive.enabled",True)
spark.conf.set("spark.sql.adaptive.skewJoin.enabled",True)
spark.conf.set("spark.sql.adaptive.skewjoin.skewPatittionThresholdInBytes","256mb")
